In [1]:
import os
import sqlite3
import pandas as pd
from train_models import load_data_from_db, train_models
from predict_cuisine import predict_cuisine
from db_setup import init_db

print("Все модули успешно импортированы ✅")


Все модули успешно импортированы ✅


In [3]:
init_db()

📥 Таблица пуста — загружаем данные из JSON...
✅ Загружено 25455 записей из train.json в таблицу 'recipes'.
✅ Загружено 6364 записей из test.json в таблицу 'recipes'.
✅ База данных готова: recipes.db


In [5]:
train_models()

📊 Загружено из БД: 31819 записей
  → Обучение: 25455
  → Тест: 6364
✅ Пример текста:
garlic paste potatoes onions dashi gochujang base zucchini fresh mushrooms water soft tofu bean curd

Создание TF-IDF векторизатора (1–3-граммы)...
Размер словаря: 30000
Матрица обучения: (25455, 30000)
Матрица теста: (6364, 30000)
✅ Векторизатор сохранён: vectorizer.pkl

🧠 Обучение Multinomial Naive Bayes...

📈 Multinomial Naive Bayes — результаты:
  Accuracy: 0.7142
  Precision: 0.7352
  Recall: 0.7142
  F1-score: 0.7006

⚙️ Обучение Logistic Regression...


C:\Users\a.kozak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



📈 Logistic Regression — результаты:
  Accuracy: 0.7737
  Precision: 0.7735
  Recall: 0.7737
  F1-score: 0.7686

🌲 Обучение Random Forest...

📈 Random Forest — результаты:
  Accuracy: 0.7220
  Precision: 0.7424
  Recall: 0.7220
  F1-score: 0.7014

💡 Обучение LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.351442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 221489
[LightGBM] [Info] Number of data points in the train set: 25455, number of used features: 6683
[LightGBM] [Info] Start training from score -4.474787
[LightGBM] [Info] Start training from score -3.871790
[LightGBM] [Info] Start training from score -3.244944
[LightGBM] [Info] Start training from score -2.705108
[LightGBM] [Info] Start training from score -3.956403
[LightGBM] [Info] Start training from score -2.711592
[LightGBM] [Info] Start training from score -3.5

C:\Users\a.kozak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



📈 LightGBM — результаты:
  Accuracy: 0.7745
  Precision: 0.7727
  Recall: 0.7745
  F1-score: 0.7689

🤝 Обучение ансамбля (VotingClassifier)...


C:\Users\a.kozak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



📈 Ensemble — результаты:
  Accuracy: 0.7855
  Precision: 0.7846
  Recall: 0.7855
  F1-score: 0.7798

🏆 Лучшая модель: Ensemble
✅ Модель сохранена: model.pkl


(VotingClassifier(estimators=[('lr',
                               LogisticRegression(C=10, max_iter=3000,
                                                  multi_class='multinomial',
                                                  random_state=42,
                                                  solver='saga')),
                              ('lgb',
                               LGBMClassifier(max_depth=16, n_estimators=1000,
                                              n_jobs=-1, num_leaves=256,
                                              random_state=42))],
                  n_jobs=-1, voting='soft'),
 TfidfVectorizer(max_df=0.9, max_features=30000, min_df=2, ngram_range=(1, 3),
                 sublinear_tf=True))

In [6]:
predict_cuisine("soy, sauce, noodles, ginger, sesame, oil")

✅ Модель и векторизатор загружены.

🥘 Предсказанная кухня: chinese
🔢 Топ-5 вероятностей:
  chinese: 0.9316
  japanese: 0.0437
  korean: 0.0209
  vietnamese: 0.0019
  filipino: 0.0008
💾 Предсказание сохранено в БД.

🕘 Последние предсказания:
🧾 2025-11-04 10:24:37 | chinese (0.9316) ← soy, sauce, noodles, ginger, sesame, oil
🧾 2025-11-04 10:24:10 | italian (0.9997) ← pasta, tomato, parmesan, olive, oil
🧾 2025-11-04 10:23:43 | italian (0.6122) ← garlic, tofu, mushrooms, zucchini
🧾 2025-11-04 10:22:46 | mexican (0.9012) ← beef, onions, tomatoes, cumin
🧾 2025-11-04 10:22:26 | mexican (0.9811) ← beef, onions, tomatoes, cumin, chili powder, garlic


In [12]:
import sqlite3

DB_PATH = "recipes.db"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("SELECT * FROM predictions")
rows = cursor.fetchall()

for i, row in enumerate(rows, 1):
    print(f"\n📄 Запись {i}:")
    print("id:", row[0])
    print("input_text:", row[1])
    print("predicted_cuisine:", row[2])
    print("probability:", row[3])
    print("timestamp:", row[4])

conn.close()



📄 Запись 1:
id: 1
input_text: beef, onions, tomatoes, cumin, chili powder, garlic
predicted_cuisine: mexican
probability: 0.9811178560683911
timestamp: 2025-11-04 10:18:03

📄 Запись 2:
id: 2
input_text: beef, onions, tomatoes, cumin, chili powder, garlic
predicted_cuisine: mexican
probability: 0.9811178560683911
timestamp: 2025-11-04 10:20:48

📄 Запись 3:
id: 3
input_text: beef, onions, tomatoes, cumin, chili powder, garlic
predicted_cuisine: mexican
probability: 0.9811178560683911
timestamp: 2025-11-04 10:22:26

📄 Запись 4:
id: 4
input_text: beef, onions, tomatoes, cumin
predicted_cuisine: mexican
probability: 0.901210649807026
timestamp: 2025-11-04 10:22:46

📄 Запись 5:
id: 5
input_text: garlic, tofu, mushrooms, zucchini
predicted_cuisine: italian
probability: 0.6121688229820232
timestamp: 2025-11-04 10:23:43

📄 Запись 6:
id: 6
input_text: pasta, tomato, parmesan, olive, oil
predicted_cuisine: italian
probability: 0.9997287924872548
timestamp: 2025-11-04 10:24:10

📄 Запись 7:
id: 7


In [17]:
!{sys.executable} -m streamlit --version


Streamlit, version 1.51.0


In [ ]:
import sys, subprocess, os

# Отключаем все интерактивные запросы Streamlit
os.environ["STREAMLIT_BROWSER_GATHER_USAGE_STATS"] = "false"
os.environ["STREAMLIT_SERVER_HEADLESS"] = "true"  # важно для Jupyter
os.environ["PYTHONUNBUFFERED"] = "1"

# Запускаем Streamlit в фоне, чтобы не блокировать Jupyter
process = subprocess.Popen(
    [sys.executable, "-m", "streamlit", "run", "app.py", "--server.port", "8501"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print("🚀 Streamlit запущен на http://localhost:8501")

# Читаем только первые строки лога, чтобы убедиться, что сервер поднялся
for _ in range(20):
    line = process.stdout.readline()
    if not line:
        break
    print(line.strip())


🚀 Streamlit запущен на http://localhost:8501

You can now view your Streamlit app in your browser.

Local URL: http://localhost:8501
Network URL: http://172.26.240.8:8501
External URL: http://213.184.224.165:8501

